A simple ad-hoc implementation for computing point group matrices based on libmsysm. Currently, only groups without reflection are supported because the python interface of libmsysm is quite inconvenient for implementing such functions. It is essentially constructing inputs to cheat a rigid high-level interface to recover a low-level interface. Exposing the low-level functionalities directly would be much more elegant.

Author: Hangrui Bi
No rights reserved.

# libmsysm

In [ ]:
import libmsym as msym
import numpy as np

In [ ]:
gen_elements = [msym.Element(name = "C", coordinates = [1.443524, 0.0,0.0])]
elements = [msym.Element(name = "C", coordinates = map(float, (0, 0, 0)))] # symmetry center
ctx = msym.Context(elements = elements, point_group='D4')
gen_elements = ctx.generate_elements(gen_elements)
for item in gen_elements:
    print(item.coordinates)

In [ ]:
ctx.symmetry_operations 

# Wrappers

In [ ]:
import pdb
def trajectory(points, group, center=np.array([0.,0.,0.])):
    eps = 1e-12
    points = points + np.random.randn(*points.shape)*eps
    # ad hoc method to bypass equivalence set
    assert(len(points.shape) == 2)
    n_points = points.shape[0]
    elements = [msym.Element(name = "C")] 
    gen_elements = [msym.Element(name = "C", coordinates = list(points[i]-center)) for i in range(n_points)]
    # notice that for C and D, the molecules zero-centerd along the symmetry axis
    with msym.Context(elements=elements, point_group=group) as ctx:
        ctx.set_thresholds(permutation=1e-15)
        order = len(ctx.symmetry_operations)
        traj = []
        for i in range(n_points):
            offset = center
            if 'C' in group:
                offset = offset + (points[i]-center)*np.array([0., 0., 1.])
            tmp = [np.array(item.coordinates) + offset for item in ctx.generate_elements([gen_elements[i]])]
            traj += tmp
    result = []
    for i in range(order):
        result += [np.stack([traj[j*order+i] for j in range(n_points)])]
    return result

def rotationMatrices(group):
    """ Outputs the rotaition matrices given a group """
    points = np.eye((3))
    result = trajectory(points, group)
    return result

# Test

In [ ]:
group = 'C4' # choose from [Cn, Dn, T, O, I], currently reflections are not supported 
atol = 1e-6

In [ ]:
# preserves center
center = np.random.randn(1, 3)
tmp = trajectory(center, group, center[0])
for item in tmp:
    assert np.allclose(item, center, atol=atol)

In [ ]:
# preserves norm
center = np.random.randn(1, 3)
point = np.random.randn(1, 3)
tmp = trajectory(point, group, center[0])
for item in tmp:
    assert np.allclose(np.linalg.norm(item-center), np.linalg.norm(point[0]-center), atol=atol)

In [ ]:
# closed under inversion
tmp = rotationMatrices(group)
order = len(tmp)
for i in range(order):
    flag = False
    for j in range(order):
        if np.allclose(tmp[i]@tmp[j], np.eye(3), atol=atol):
            flag = True
    assert flag, i

In [ ]:
# closed under multiplication
tmp = rotationMatrices(group)
order = len(tmp)
for i in range(order):
    for j in range(order):
        flag = False
        for k in range(order):
            if np.allclose(tmp[i]@tmp[j], tmp[k], atol=atol):
                flag = True
        assert flag, (i, j)